In [ ]:
import pandas as pd
import librosa, librosa.display
import os
import numpy as np
import scipy, matplotlib.pyplot as plt
import math
import IPython.display as ipd
import scipy
import torch
import torch
from torch.utils.data import Dataset

In [ ]:
# define the number of minimum and maximum records per speaker
MAX_RECORDS_PER_SPEAKER = 50
MIN_RECORDS_PER_SPEAKER = 40

In [ ]:
tsv_filename = 'train' # choose one of 'train' or 'dev'
filepath = tsv_filename + '.tsv'

In [ ]:
# read validated.tsv file from Mozilla Voice Common dataset
validated_tsv_file_path = '<path>/<tp>/datasets/en'
validated_df = pd.read_csv(os.path.join(validated_tsv_file_path, filepath), sep='\t')
validated_df.head(6)

In [ ]:
print("Data frame size:", validated_df.shape)
print("Number of unique speakers:", len(validated_df['client_id'].unique()))

In [ ]:
valid_records = {}
counter = 0
for group in validated_df.groupby('client_id'):
    group_size = len(group[1])
    # keep only speakers with a minimum number of audio samples
    if group_size >= MIN_RECORDS_PER_SPEAKER:
        valid_records[group[0]] = counter
        counter += 1

In [ ]:
valid_records

In [ ]:
print("Number of valid records:", len(valid_records))

In [ ]:
valid_df = pd.DataFrame()

for client_id in valid_records.keys():
    group_df = validated_df[validated_df['client_id'] == client_id]
    if len(group_df) > MAX_RECORDS_PER_SPEAKER:
        print("More than {0} records for speaker: {1}".format(MAX_RECORDS_PER_SPEAKER, client_id))
        group_df = group_df.sample(n = MAX_RECORDS_PER_SPEAKER)
    
    valid_df = valid_df.append(group_df)

valid_df = valid_df.reset_index(drop=True)

In [ ]:
print("Valid dataframe size:", valid_df.shape)
print("Number of unique speakers:", len(valid_df['client_id'].unique()))

In [ ]:
# map client hashs to class ids
valid_df['client_id'] = valid_df['client_id'].map(lambda x: valid_records[x])
valid_df.tail()

In [ ]:
# save the valid subset for processing
valid_df.to_csv(tsv_filename + '_valid.tsv', index=False, sep='\t', header=True)